In [1]:
import tensorflow as tf
import numpy as np
import utils

__author__="Olivares Castillo José Luis"

print("TensorFlow",tf.VERSION)
if tf.test.gpu_device_name():
    print("GPU disponible")

tf.logging.set_verbosity(tf.logging.INFO)

/home/olivares/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


TensorFlow 1.7.0


In [2]:
src_lex, trg_lex = utils.get_lexicon("en-it.test")

In [3]:
len(set(src_lex)), len(set(trg_lex))

(1500, 1849)

In [4]:
src_matrix = utils.open_file("en-crawl")

In [5]:
words_src, source_vec = utils.read(src_matrix,vocabulary=src_lex)

In [6]:
len(words_src)

1387

In [7]:
for i in src_lex:
    if i not in words_src:
        print(i)

suharto
bogotá
bogotá
sorbonne
billingham
walesa
nazarbayev
papadopoulos
antioquia
togolese
togolese
scowcroft
unrwa
tegucigalpa
rafah
pilger
bessarabia
valenciennes
méndez
vukovar
csps
radovan
michèle
allawi
kursk
yalta
núñez
ceausescu
interreg
vélez
vercelli
kfor
saakashvili
jörg
beazley
ingushetia
prizren
isfahan
lumumba
hélène
agusta
unclos
guterres
jiabao
kumaratunga
bavarians
elmar
ratko
eeurope
beckenbauer
tamayo
genk
sète
rosati
ncbs
chandrika
véronique
romanies
malpensa
jaruzelski
raynal
unodc
antonello
brueghel
caretta
jaurès
inés
mossadeq
schreyer
lahoud
cpmp
nanterre
daghestan
sudeten
lindh
einaudi
kouchner
tindouf
lyudmila
fassa
macra
megawati
izetbegovic
bouteflika
hristo
ngawang
rovaniemi
fyrom
michelis
igad
nlg
ogoni
heysel
matera
ravalomanana
idriss
andreotti
bassam
ekaterinburg
gaborone
tornio
kostunica
clarín
wilfredo
karamanlis
mendieta
atocha
gbagbo
euskadi
rehn
gaullists
kabylie
thüringen
katyn
ecevit
